# Libraries and dataset of S&P500 for 1h timeframe

In [ ]:

import numpy as np
import pandas as pd
import math
import os
try:
    from google.colab import files

    if not os.path.exists("/content/df-SP500_2018-2019.ftr"):
      !gdown --id 1npRcr-8tpS_4ym6qd9Fm7BhKGqIp5-bN  # #https://drive.google.com/file/d/1npRcr-8tpS_4ym6qd9Fm7BhKGqIp5-bN/view?usp=sharing
except:
    pass

# Init parameters

In [ ]:
#@title Init Parameters
INIT_CAPITAL = 1000000  #@param {type:"integer"}
POSITION_SIZE_PERCENT = 0.05  #@param {type:"number"}
TAKE_PROFIT = 0.05  #@param {type:"number"}
STOP_LOSS = 0.02  #@param {type:"number"}
COMISSION = 0.001  #@param {type:"number"}


# Backtesting functions

In [ ]:
PORTFOLIO = {}
CURRENT_CASH = INIT_CAPITAL
WINNING_TRADES = 0
LOSING_TRADES = 0
CURRENT_TRADE_ID = 1

SEED = 123  # To fix random for results repeating
np.random.seed(SEED)


def calc_trade(ticker_code, portfolio, position_ohlc, operation, exit_type=''):
    global WINNING_TRADES
    global LOSING_TRADES
    is_sale = operation == 'SELL'

    buy_volume = portfolio['price'] * portfolio['size']
    buy_volume_comission = buy_volume * (1 + COMISSION)

    print(portfolio['id'], 'BUY', position_ohlc['datetime'].strftime('%Y-%m-%d %H:%M'), ticker_code, f'${buy_volume:.2f}')
    if is_sale:
        held_days = (position_ohlc['datetime'] - portfolio['datetime']).days
        sel_volume = portfolio['size'] * position_ohlc['close']
        sel_volume_comission = sel_volume * (1 - COMISSION)
        profit = sel_volume_comission - buy_volume_comission
        if profit > 0:
            WINNING_TRADES += 1
        elif profit < 0:
            LOSING_TRADES += 1
        print(f'{portfolio["id"]} SELL {exit_type}',
              position_ohlc['datetime'].strftime('%Y-%m-%d %H:%M'), ticker_code, f'profit ${profit:.2f}, held',
              (position_ohlc['datetime'] - portfolio['datetime']).days, 'days')


def check_conditions_to_exit(df, are_sell_all=False):
    global CURRENT_CASH
    if not PORTFOLIO:
        return
    sold_tickers = []
    for ticker_code in PORTFOLIO:
        ticker_df = df[df['ticker'] == ticker_code]
        if not ticker_df.empty:
            position_ohlc = ticker_df.iloc[0]
            buy_price = PORTFOLIO[ticker_code]['price']
            is_take_profit = position_ohlc['high'] > buy_price and (position_ohlc['high'] - buy_price) / buy_price >= TAKE_PROFIT
            is_stop_loss = position_ohlc['low'] < buy_price and (buy_price - position_ohlc['low']) / buy_price >= STOP_LOSS
            if is_take_profit or is_stop_loss or are_sell_all:
                sell_volume = PORTFOLIO[ticker_code]['size'] * position_ohlc['close']
                CURRENT_CASH += sell_volume - sell_volume * COMISSION
                calc_trade(ticker_code, PORTFOLIO[ticker_code], position_ohlc, 'SELL', 'TP' if is_take_profit else (
                    'SL' if not are_sell_all else 'CLOSE'))
                sold_tickers.append(ticker_code)
        elif are_sell_all:
            buy_price = PORTFOLIO[ticker_code]['price']
            buy_volume = buy_price * PORTFOLIO[ticker_code]['size'] * (1 + COMISSION)
            sell_volume = buy_volume
            CURRENT_CASH += sell_volume - sell_volume * COMISSION
            position_ohlc = {'datetime': df['datetime'].max(), 'close': buy_price}
            calc_trade(ticker_code, PORTFOLIO[ticker_code], position_ohlc, 'SELL', 'CLOSE')
            sold_tickers.append(ticker_code)
    for ticker_code in sold_tickers:
        del PORTFOLIO[ticker_code]


def get_value_of_tickers_held():
    value = 0
    for ticker_code in PORTFOLIO:
        value += PORTFOLIO[ticker_code]['size'] * PORTFOLIO[ticker_code]['price']
    return value


def prepare_tickers_list_to_buy(tickers_held, tickers_on_the_exchange):
    tickers_to_buy = list(filter(lambda ticker_code: ticker_code not in tickers_held, tickers_on_the_exchange))
    tickers_to_buy = np.array(tickers_to_buy)
    np.random.shuffle(tickers_to_buy)
    return tickers_to_buy


def buy_tickers(df):
    global CURRENT_CASH
    global ALL_TRADES
    global CURRENT_TRADE_ID
    current_value = get_value_of_tickers_held()
    full_value = CURRENT_CASH + current_value
    max_position_value = full_value * POSITION_SIZE_PERCENT
    if CURRENT_CASH > max_position_value:
        number_tickers_to_buy = math.floor(CURRENT_CASH / max_position_value)
        tickers_on_the_exchange = df['ticker'].unique()
        tickers_held = PORTFOLIO.keys()
        tickers_to_buy = prepare_tickers_list_to_buy(tickers_held, tickers_on_the_exchange)
        for idx in range(min(number_tickers_to_buy, len(tickers_to_buy))):
            ticker_code = tickers_to_buy[idx]
            ticker_df = df[df['ticker'] == ticker_code]
            if not ticker_df.empty and max_position_value and ticker_df.iloc[0]['close']:
                position_ohlc = ticker_df.iloc[0]
                if max_position_value <= position_ohlc['close']:
                    continue
                PORTFOLIO[ticker_code] = {
                    'id': CURRENT_TRADE_ID,
                    'price': position_ohlc['close'],
                    'size': math.floor(max_position_value / position_ohlc['close']),
                    'datetime': position_ohlc['datetime']
                }
                CURRENT_TRADE_ID += 1
                buy_volume = PORTFOLIO[ticker_code]['price'] * PORTFOLIO[ticker_code]['size']
                CURRENT_CASH += (-buy_volume) - buy_volume * COMISSION
                calc_trade(ticker_code, PORTFOLIO[ticker_code], position_ohlc, 'BUY')


def get_current_value_of_tickers_held(df_for_datetime):
    value = 0
    for ticker_code in PORTFOLIO:
        df_ticker_price = df_for_datetime[df_for_datetime['ticker'] == ticker_code]
        if not df_ticker_price.empty:
            ticker_price = df_ticker_price['close'].iloc[-1]
        else:
            ticker_price = PORTFOLIO[ticker_code]['price']
        value += PORTFOLIO[ticker_code]['size'] * ticker_price
    return value


def backtest(df):
    check_conditions_to_exit(df)
    buy_tickers(df)

# Backtest

In [ ]:
df_sp = pd.read_feather('./df-SP500_2018-2019.ftr')
df_sp.drop(['exchange', 'full_ticker'], axis=1, inplace=True)
df_sp.dropna(subset=['close'], inplace=True)
df_sp.sort_values(by=['datetime', 'ticker'], inplace=True)
df_sp.groupby('datetime').apply(backtest)
df_for_last_datetime = df_sp[df_sp['datetime'] == df_sp['datetime'].max()]
check_conditions_to_exit(df_for_last_datetime, True)
current_close_value = get_current_value_of_tickers_held(df_for_last_datetime)
full_value = CURRENT_CASH + current_close_value

trades_num = WINNING_TRADES + LOSING_TRADES
trades_ratio = WINNING_TRADES / (WINNING_TRADES + LOSING_TRADES)

print(f'Net profit ${(full_value - INIT_CAPITAL):.2f}')
print(f'Trades: {trades_num}, ratio {trades_ratio:.2f}, winning: {WINNING_TRADES}, losing: {LOSING_TRADES}')

Streaming output truncated to the last 5000 lines.
283 SELL SL 2018-03-22 15:00 STX profit $-1189.07, held 2 days
287 BUY 2018-03-22 15:00 VFC $50604.75
287 SELL SL 2018-03-22 15:00 VFC profit $-1173.39, held 1 days
290 BUY 2018-03-22 15:00 SPGI $50648.00
290 SELL SL 2018-03-22 15:00 SPGI profit $-1168.83, held 0 days
291 BUY 2018-03-22 15:00 RCL $50270.10
292 BUY 2018-03-22 15:00 HST $50355.00
293 BUY 2018-03-22 15:00 ORLY $50169.24
294 BUY 2018-03-22 15:00 SWKS $50279.90
295 BUY 2018-03-22 15:00 TSCO $50330.28
291 BUY 2018-03-22 19:00 RCL $50270.10
291 SELL SL 2018-03-22 19:00 RCL profit $-1095.84, held 0 days
296 BUY 2018-03-22 19:00 PM $50289.75
268 BUY 2018-03-23 08:00 PFE $50273.37
268 SELL SL 2018-03-23 08:00 PFE profit $-991.18, held 14 days
297 BUY 2018-03-23 08:00 INTC $50239.71
277 BUY 2018-03-23 13:00 AOS $50666.04
277 SELL SL 2018-03-23 13:00 AOS profit $-1052.40, held 8 days
292 BUY 2018-03-23 13:00 HST $50355.00
292 SELL SL 2018-03-23 13:00 HST profit $-1098.71, held 0 d